In [1]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# plotly
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# !pip install wandb -qqq
import wandb
# wandb.login()
api = wandb.Api()
import os

os.environ["WANDB_NOTEBOOK_NAME"] = "jup_res"


# selected_tag = 'Pred'

runs = api.runs("hokarami/TEDAM3")


In [3]:
import pandas as pd 


# Project is specified by <entity/project-name>
def dl_runs(all_runs, selected_tag=None):




    summary_list, config_list, name_list = [], [], []
    for run in runs: 

        if (selected_tag not in run.tags) and (selected_tag is not None):
            continue
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files 
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        

        config_list.append(
            {k: v for k,v in run.config.items()
            if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)

    runs_df = pd.DataFrame({
        "summary": summary_list,
        "config": config_list,
        "name": name_list
        })

    # runs_df.to_csv("project.csv")
    return runs_df

In [4]:
# q = runs_df['name'].str.contains('\[R2-')
# runs_df = runs_df[q]


runs_df = dl_runs(runs, selected_tag=None)

runs_df
runs_df.iloc[0].summary.keys()
runs_df.iloc[0].config.keys()

,summary,config,name
0,"{'Valid-CIF/LL-#events': -1.7310217276704225, ...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[V1]1256407
1,{'Train-pred_label/f1-binary': 0.8167075389605...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[V1]1251417
2,"{'Train-NLL/#events': -1, 'Train-NextTime/msae...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[V1]1256164
3,{'Best-Test-pred_label/f1-binary': 0.556836902...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[V1]1253052
4,"{'_timestamp': 1674252173.1183088, 'Train-NLL/...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[V1]1257487
...,...,...,...
579,"{'Test-CIF/NLL-#events': -0.2362936914823396, ...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[CIF-mc1]TE__shpmark-1153303
580,"{'Train-CIF/NLL-#events': -0.4912015046472573,...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[CIF-mc1]TE__shpmark-1153303
581,{'Test-NextType(ML)/recall-weighted': 0.841599...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[CIF-mc1]TE__shpmark-1153303
582,{'Test-NextType(ML)/precision-weighted': 0.502...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[CIF-mc1]TE__shpmark-1153303


dict_keys(['Valid-CIF/LL-#events', 'Valid-NextType(ML)/auc-PR-weighted', 'Valid-NextType(ML)/precision-weighted', 'Valid-CIF/NLL', 'Valid-CIF/#events', 'Valid-NextTime/msae', 'Valid-NextTime/rmse_norm', 'Valid-NextType(ML)/f1-weighted', 'Valid-NextType(ML)/recall-weighted', 'Valid-NextType(ML)/auc-ovo-weighted', '_step', '_runtime', '_timestamp', 'Valid-NextTime/RMSE'])

dict_keys(['w', 'lr', 'log', 'mod', 'per', 'cuda', 'data', 'date', 'demo', 'prof', 'INPUT', 'epoch', 'split', 'state', 'w_pos', 'wandb', 'ES_pat', 'OUTPUT', 'device', 'freeze', 'run_id', 'smooth', 'te_d_k', 'te_d_v', 'w_time', 'dataset', 'int_dec', 'setting', 'w_class', 'w_event', 'run_name', 'run_path', 'te_d_rnn', 'time_enc', 'TE_config', 'event_enc', 'next_mark', 'num_demos', 'num_marks', 'num_types', 'pos_alpha', 'te_d_mark', 'te_d_time', 'te_n_head', 'type_loss', 'CIF_config', 'DAM_config', 'batch_size', 'data_label', 'num_states', 'pos_weight', 'str_config', 'te_d_inner', 'te_dropout', 'testloader', 'demo_config', 'mark_detach', 'te_n_layers', 'test_center', 'trainloader', 'user_prefix', 'validloader', 'w_pos_label', 'label_config', 'sample_label', 'weight_decay', 'hparams2write', 'balanced_batch', 'label_loss_fun', 'pred_loss_func', 'w_sample_label', 'next_time_config', 'next_type_config', 'transfer_learning'])

# Pred

In [5]:
df_filt = dl_runs(runs, selected_tag='V1')

In [6]:
i_groupby_row =['dataset','setting','test_center']
i_groupby_col = ['event_enc','mod']
i_groupby_col=['OUTPUT','INPUT']
i_agg = ['split']
i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC','Best-Test-pred_label/AUROC']
# i_metric = ['Test-CIF/NLL-#events']


#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]
df




a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(['mean','std']).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:str(round(x.mean(),2))  + ' (' + str(round(x.std(),2))  +')'  ) #  

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_groupby_row,columns=i_groupby_col)



# a1.to_csv("aaa.csv")

,dataset,setting,test_center,OUTPUT,INPUT,split,Best-Test-pred_label/f1-binary,Best-Test-pred_label/AUPRC,Best-Test-pred_label/AUROC
0,/scratch/hokarami/data_tedam/p12/,mc1,0,none-label,TEDAM,3,0.452316,0.495551,0.853303
1,/scratch/hokarami/data_tedam/p19/,mc1,0,single-mark,TE,1,NaN,NaN,NaN
2,/scratch/hokarami/data_tedam/p19/,mc1,0,none-label,DAM,0,0.556837,0.725446,0.911587
3,/scratch/hokarami/data_tedam/p19/,mc1,0,none-label,TEDAM,0,0.654545,0.773746,0.932928
4,/scratch/hokarami/data_tedam/p19/,mc1,0,single-mark,TEDAM,0,NaN,NaN,NaN
5,/scratch/hokarami/data_tedam/p19/,mc1,0,ml,TEDAM,0,NaN,NaN,NaN
6,/scratch/hokarami/data_tedam/p19/,mc1,0,single-mark,TE,0,NaN,NaN,NaN
7,/scratch/hokarami/data_tedam/p12/,mc1,0,none-label,DAM,3,0.477454,0.460856,0.850558
8,/scratch/hokarami/data_tedam/p12/,mc1,0,ml,TEDAM,3,NaN,NaN,NaN
9,/scratch/hokarami/data_tedam/p12/,mc1,0,single-mark,TEDAM,3,NaN,NaN,NaN


Best-Test-pred_label/f1-binary  \
dataset                           setting test_center OUTPUT      INPUT                                  
/scratch/hokarami/data_tedam/p12/ mc1     0           ml          TEDAM                      nan (nan)   
                                                      none-label  DAM                      0.47 (0.01)   
                                                                  TEDAM                    0.54 (0.06)   
                                                      single-mark TE                         nan (nan)   
                                                                  TEDAM                      nan (nan)   
/scratch/hokarami/data_tedam/p19/ mc1     0           ml          TEDAM                      nan (nan)   
                                                      none-label  DAM                       0.56 (nan)   
                                                                  TEDAM                     0.65 (nan)   
                                                      single-mark TE                         nan (nan)   
                                                                  TEDAM                      nan (nan)   

                                                                        Best-Test-pred_label/AUPRC  \
dataset                           setting test_center OUTPUT      INPUT                              
/scratch/hokarami/data_tedam/p12/ mc1     0           ml          TEDAM                  nan (nan)   
                                                      none-label  DAM                  0.49 (0.03)   
                                                                  TEDAM                 0.6 (0.09)   
                                                      single-mark TE                     nan (nan)   
                                                                  TEDAM                  nan (nan)   
/scratch/hokarami/data_tedam/p19/ mc1     0           ml          TEDAM                  nan (nan)   
                                                      none-label  DAM                   0.73 (nan)   
                                                                  TEDAM                 0.77 (nan)   
                                                      single-mark TE                     nan (nan)   
                                                                  TEDAM                  nan (nan)   

                                                                        Best-Test-pred_label/AUROC  
dataset                           setting test_center OUTPUT      INPUT                             
/scratch/hokarami/data_tedam/p12/ mc1     0           ml          TEDAM                  nan (nan)  
                                                      none-label  DAM                  0.86 (0.01)  
                                                                  TEDAM                0.91 (0.04)  
                                                      single-mark TE                     nan (nan)  
                                                                  TEDAM                  nan (nan)  
/scratch/hokarami/data_tedam/p19/ mc1     0           ml          TEDAM                  nan (nan)  
                                                      none-label  DAM                   0.91 (nan)  
                                                                  TEDAM                 0.93 (nan)  
                                                      single-mark TE                     nan (nan)  
                                                                  TEDAM                  nan (nan)

Best-Test-pred_label/f1-binary  \
OUTPUT                                                                            ml   
INPUT                                                                          TEDAM   
dataset                           setting test_center                                  
/scratch/hokarami/data_tedam/p12/ mc1     0                                nan (nan)   
/scratch/hokarami/data_tedam/p19/ mc1     0                                nan (nan)   

                                                                    \
OUTPUT                                                  none-label   
INPUT                                                          DAM   
dataset                           setting test_center                
/scratch/hokarami/data_tedam/p12/ mc1     0            0.47 (0.01)   
/scratch/hokarami/data_tedam/p19/ mc1     0             0.56 (nan)   

                                                                    \
OUTPUT                                                               
INPUT                                                        TEDAM   
dataset                           setting test_center                
/scratch/hokarami/data_tedam/p12/ mc1     0            0.54 (0.06)   
/scratch/hokarami/data_tedam/p19/ mc1     0             0.65 (nan)   

                                                                              \
OUTPUT                                                single-mark              
INPUT                                                          TE      TEDAM   
dataset                           setting test_center                          
/scratch/hokarami/data_tedam/p12/ mc1     0             nan (nan)  nan (nan)   
/scratch/hokarami/data_tedam/p19/ mc1     0             nan (nan)  nan (nan)   

                                                      Best-Test-pred_label/AUPRC  \
OUTPUT                                                                        ml   
INPUT                                                                      TEDAM   
dataset                           setting test_center                              
/scratch/hokarami/data_tedam/p12/ mc1     0                            nan (nan)   
/scratch/hokarami/data_tedam/p19/ mc1     0                            nan (nan)   

                                                                    \
OUTPUT                                                  none-label   
INPUT                                                          DAM   
dataset                           setting test_center                
/scratch/hokarami/data_tedam/p12/ mc1     0            0.49 (0.03)   
/scratch/hokarami/data_tedam/p19/ mc1     0             0.73 (nan)   

                                                                               \
OUTPUT                                                            single-mark   
INPUT                                                       TEDAM          TE   
dataset                           setting test_center                           
/scratch/hokarami/data_tedam/p12/ mc1     0            0.6 (0.09)   nan (nan)   
/scratch/hokarami/data_tedam/p19/ mc1     0            0.77 (nan)   nan (nan)   

                                                                  \
OUTPUT                                                             
INPUT                                                      TEDAM   
dataset                           setting test_center              
/scratch/hokarami/data_tedam/p12/ mc1     0            nan (nan)   
/scratch/hokarami/data_tedam/p19/ mc1     0            nan (nan)   

                                                      Best-Test-pred_label/AUROC  \
OUTPUT                                                                        ml   
INPUT                                                                      TEDAM   
dataset                           setting test_center                              
/scratch/hokarami/data_tedam/p12/ mc1     0 

# CIF

In [7]:
df_filt = dl_runs(runs, selected_tag='CIF')

df_filt.iloc[0].config.keys()

dict_keys(['w', 'lr', 'log', 'mod', 'per', 'cuda', 'data', 'date', 'demo', 'prof', 'epoch', 'split', 'state', 'w_pos', 'wandb', 'ES_pat', 'device', 'freeze', 'run_id', 'smooth', 'te_d_k', 'te_d_v', 'w_time', 'dataset', 'int_dec', 'setting', 'w_class', 'w_event', 'run_name', 'run_path', 'te_d_rnn', 'time_enc', 'TE_config', 'event_enc', 'next_mark', 'num_demos', 'num_marks', 'num_types', 'pos_alpha', 'te_d_mark', 'te_d_time', 'te_n_head', 'type_loss', 'CIF_config', 'DAM_config', 'batch_size', 'data_label', 'num_states', 'pos_weight', 'str_config', 'te_d_inner', 'te_dropout', 'testloader', 'demo_config', 'mark_detach', 'te_n_layers', 'test_center', 'trainloader', 'user_prefix', 'validloader', 'w_pos_label', 'label_config', 'sample_label', 'weight_decay', 'hparams2write', 'balanced_batch', 'label_loss_fun', 'pred_loss_func', 'w_sample_label', 'next_time_config', 'next_type_config', 'transfer_learning'])

In [9]:
i_groupby_row =['dataset','setting',]
i_groupby_col = ['state']
i_agg = ['test_center','split']
# i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC']
i_metric = ['Test-CIF/NLL-#events']


#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)#.dropna(axis=1,how='all')
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]
df





a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(['mean','std']).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:str(round(x.mean(),3))  + ' (' + str(round(x.std(),3))  +')' )

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_groupby_row,columns=i_groupby_col)



# a1.to_csv("aaa.csv")

,dataset,setting,state,test_center,split,Test-CIF/NLL-#events
0,/scratch/hokarami/data_tedam/p19/,mc1,True,1,2,0.607923
1,/scratch/hokarami/data_tedam/p19/,mc1,True,1,0,0.698569
2,/scratch/hokarami/data_tedam/p19/,mc1,True,1,1,0.785800
3,/scratch/hokarami/data_tedam/p19/,mc1,True,1,4,0.705369
4,/scratch/hokarami/data_tedam/p19/,mc1,True,1,3,0.749281
...,...,...,...,...,...,...
135,/scratch/hokarami/data_tedam/p19/,sc,False,0,2,0.667987
136,/scratch/hokarami/data_tedam/p19/,sc,False,0,1,0.641528
137,/scratch/hokarami/data_tedam/p19/,sc,False,0,0,0.691973
138,/scratch/hokarami/data_tedam/p19/,sc,False,0,3,0.750535


Test-CIF/NLL-#events
dataset                           setting state                     
/scratch/hokarami/data_tedam/p12/ mc1     False         -0.4 (0.138)
                                          True         -0.797 (0.13)
                                  mc2     False       -0.312 (0.083)
                                          True        -0.781 (0.046)
                                  sc      False        0.075 (0.073)
                                          True        -0.329 (0.111)
/scratch/hokarami/data_tedam/p19/ mc1     False          0.808 (0.2)
                                          True         0.582 (0.146)
                                  mc2     False        0.988 (0.148)
                                          True         0.857 (0.164)
                                  sc      False         0.94 (0.275)
                                          True          0.68 (0.299)

Test-CIF/NLL-#events                
state                                                    False            True
dataset                           setting                                     
/scratch/hokarami/data_tedam/p12/ mc1             -0.4 (0.138)   -0.797 (0.13)
                                  mc2           -0.312 (0.083)  -0.781 (0.046)
                                  sc             0.075 (0.073)  -0.329 (0.111)
/scratch/hokarami/data_tedam/p19/ mc1              0.808 (0.2)   0.582 (0.146)
                                  mc2            0.988 (0.148)   0.857 (0.164)
                                  sc              0.94 (0.275)    0.68 (0.299)